In [1]:
# Import necessary libraries
import numpy as np
import random
import time
from typing import List, Tuple, Optional

In [2]:
# Tic-Tac-Toe game environment
class TicTacToe:
    def __init__(self):
        self.board = np.zeros((3, 3), dtype=int)  # 0: empty, 1: player 1, -1: player 2
    def clone(self):
        """
        Creates a deep copy of the current game state.

        Returns:
            TicTacToe: A new instance of TicTacToe with the same board state.
        """
        new_state = TicTacToe()
        new_state.board = self.board.copy()
        return new_state
    
    def reset(self):
        self.board = np.zeros((3, 3), dtype=int)

    def is_valid_move(self, x: int, y: int) -> bool:
        return self.board[x, y] == 0

    def make_move(self, x: int, y: int, player: int) -> bool:
        if self.is_valid_move(x, y):
            self.board[x, y] = player
            return True
        return False

    def check_winner(self) -> Optional[int]:
        # Check rows, columns, and diagonals
        for i in range(3):
            if abs(sum(self.board[i, :])) == 3:
                return np.sign(sum(self.board[i, :]))
            if abs(sum(self.board[:, i])) == 3:
                return np.sign(sum(self.board[:, i]))
        if abs(self.board.trace()) == 3:
            return np.sign(self.board.trace())
        if abs(np.fliplr(self.board).trace()) == 3:
            return np.sign(np.fliplr(self.board).trace())
        return None if 0 in self.board else 0  # 0: draw, None: ongoing game

    def get_available_moves(self) -> List[Tuple[int, int]]:
        return [(i, j) for i in range(3) for j in range(3) if self.board[i, j] == 0]

    def display(self):
        display_board = self.board.astype(str)
        display_board[display_board == "0"] = "."
        display_board[display_board == "1"] = "X"
        display_board[display_board == "-1"] = "O"
        print("\n".join([" ".join(row) for row in display_board]))
        print()


In [3]:
def minimax(state: TicTacToe, player: int) -> Tuple[int, Optional[Tuple[int, int]]]:
    # Check for terminal state
    winner = state.check_winner()
    if winner is not None:
        return winner, None  # Return the utility (1, -1, or 0) and no action

    available_moves = state.get_available_moves()
    best_action = None
    best_utility = -float('inf') if player == 1 else float('inf')  # Initialize best utility

    for coordinate in available_moves:
        # Make the move
        state.board[coordinate[0], coordinate[1]] = player

        # Recursive call
        utility, _ = minimax(state, -player)

        # Undo the move
        state.board[coordinate[0], coordinate[1]] = 0

        # Update best utility and action
        if (player == 1 and utility > best_utility) or (player == -1 and utility < best_utility):
            best_utility = utility
            best_action = coordinate
            
    # state.board[best_action[0], best_action[1]] = player
    return best_utility, best_action


In [4]:
def alpha_beta(state: TicTacToe, player: int, alpha: float = -float('inf'), beta: float = float('inf')) -> Tuple[int, Optional[Tuple[int, int]]]:
    # Check for terminal state
    winner = state.check_winner()
    if winner is not None:
        return winner, None  # Return utility (1, -1, or 0) and no action

    available_moves = state.get_available_moves()
    best_action = None

    if player == 1:  # Maximizing player
        best_utility = -float('inf')
        for coordinate in available_moves:
            # Make the move
            state.board[coordinate[0], coordinate[1]] = player

            # Recursive call with the opponent's turn
            utility, _ = alpha_beta(state, -player, alpha, beta)

            # Undo the move
            state.board[coordinate[0], coordinate[1]] = 0

            # Update the best utility and action
            if utility > best_utility:
                best_utility = utility
                best_action = coordinate
                
            # Update alpha and prune if possible
            alpha = max(alpha, best_utility)
            if beta <= alpha:
                break  # Beta cut-off 
    else:  # Minimizing player
        best_utility = float('inf')
        for coordinate in available_moves:
            # Make the move
            state.board[coordinate[0], coordinate[1]] = player

            # Recursive call with the opponent's turn
            utility, _ = alpha_beta(state, -player, alpha, beta)

            # Undo the move
            state.board[coordinate[0], coordinate[1]] = 0

            # Update the best utility and action
            if utility < best_utility:
                best_utility = utility
                best_action = coordinate

            # Update beta and prune if possible
            beta = min(beta, best_utility)
            if beta <= alpha:
                break  # Alpha cut-off
            
    # state.board[best_action[0], best_action[1]] = player

    return best_utility, best_action


In [5]:
from time import sleep


def evaluate_line(line: np.ndarray, current_player: int, win_length: int) -> int:
    """
    Evaluate a single line for potential:
    - More weight for lines closer to winning
    - Penalize opponent's potential wins
    """
    player_count = np.count_nonzero(line == current_player)
    opponent_count = np.count_nonzero(line == -current_player)
    empty_count = np.count_nonzero(line == 0)
    
    # Line with potential winning configuration
    if player_count > 0 and opponent_count == 0:
        potential_score = player_count * 10
        if player_count == win_length - 1:
            potential_score *= 2  # Very close to winning
        return potential_score
    
    # Line blocking opponent
    if opponent_count > 0 and player_count == 0:
        potential_score = -opponent_count * 10
        if opponent_count == win_length - 1:
            potential_score *= 2  # Block imminent threat
        return potential_score
    
    if player_count == 0 and opponent_count == 0:
        return 10
    if player_count != 0 and opponent_count != 0:
        return player_count * 10 - (opponent_count * 10)
    
    return 0  # Default return if no conditions met

def get_diagonals(board: np.ndarray, win_length: int = None) -> List[np.ndarray]:
    if win_length is None:
        win_length = board.shape[0]
    
    diagonals = []
    board_size = board.shape[0]
    
    # Main diagonals
    for k in range(-board_size + 1, board_size):
        diag = board.diagonal(k)
        if len(diag) >= win_length:
            diagonals.append(diag)
    
    # Anti-diagonals
    flipped = np.fliplr(board)
    for k in range(-board_size + 1, board_size):
        diag = flipped.diagonal(k)
        if len(diag) >= win_length:
            diagonals.append(diag)
    
    return diagonals        

def evaluate_potential_lines(state: TicTacToe, current_player: int, win_length: int) -> int:
    """Evaluate potential winning lines."""
    board = state.board
    board_size = board.shape[0]

    total_potential = 0
    
    # Rows and columns
    for i in range(board_size):
        total_potential += evaluate_line(board[i, :], current_player, win_length)
        total_potential += evaluate_line(board[:, i], current_player, win_length)
    
    # Diagonals
    for diag in get_diagonals(board, win_length):
        # print(diag)
        # state.display()
        # sleep(1)
        total_potential += evaluate_line(diag, current_player, win_length)
    
    return total_potential

def evaluate_state(state: TicTacToe, current_player: int, win_length: int = None) -> float:
    """
    Generalized evaluation function for n×n Tic-Tac-Toe board.
    """
    board = state.board

    # If win_length not specified, set it to board size
    if win_length is None:
        win_length = board.shape[0]
    
    winner = state.check_winner()
    if winner:
        return 1000 * winner if winner == current_player else -1000 * winner
    
    score = 0
    board_size = board.shape[0]
    
    # Center control
    center_indices = [board_size // 2]
    if board_size % 2 == 0:
        center_indices = [board_size // 2 - 1, board_size // 2]
    
    for x in center_indices:
        for y in center_indices:
            if board[x, y] == current_player:
                score += 30
            elif board[x, y] == -current_player:
                score -= 30
    
    # Corner control
    corner_indices = [0, board_size - 1]
    for x in corner_indices:
        for y in corner_indices:
            if board[x, y] == current_player:
                score += 20
            elif board[x, y] == -current_player:
                score -= 20
    
    # Add potential line evaluation to score
    score += evaluate_potential_lines(state, current_player, win_length)
    
    return score

def minimax_with_evaluation(
        board: np.ndarray, 
        depth: int, 
        current_player: int,
        win_length: int = None
    ) -> float:
    # Create a game state from the board
    temp_game = TicTacToe()
    temp_game.board = board.copy()
    
    # Terminal state checks
    winner = temp_game.check_winner()
    if winner is not None:
        if winner == current_player:
            return 1000
        elif winner == -current_player:
            return -1000
        else:  # Draw
            return 0
    
    # Depth limit reached
    if depth == 0:
        return evaluate_state(temp_game, current_player, win_length)
    
    # Get available moves
    moves = temp_game.get_available_moves()
    
    if current_player == 1:
        best_score = float('-inf')
        for x, y in moves:
            new_board = board.copy()
            new_board[x, y] = current_player
            score = minimax_with_evaluation(
                new_board, depth - 1, -current_player, win_length
            )
            best_score = max(best_score, score)
        return best_score
    else:
        best_score = float('-inf')
        for x, y in moves:
            new_board = board.copy()
            new_board[x, y] = current_player
            score = minimax_with_evaluation(
                new_board, depth - 1, -current_player, win_length
            )
            best_score = max(best_score, score)
        return best_score

def evaluation_based(state: TicTacToe, player: int, max_depth: int = 1) -> Tuple[int, int]:
    # Get available moves
    available_moves = state.get_available_moves()
    
    # Evaluate each possible move
    best_move = None
    best_score = float('-inf')
    
    for x, y in available_moves:
        # Try the move
        new_board = state.board.copy()
        new_board[x, y] = player
        
        # Evaluate the move using minimax
        move_score = minimax_with_evaluation(
            new_board, max_depth - 1, player
        )
        
        if move_score > best_score:
            best_score = move_score
            best_move = (x, y)
    
    # Fallback to random move if no good move found
    return 0,best_move

In [6]:
from math import sqrt, log
import random
from copy import deepcopy

class Node:
    """A node in the MCTS tree."""
    def __init__(self, state: 'TicTacToe', parent: Optional['Node'] = None, move: Optional[Tuple[int, int]] = None):
        self.state = state
        self.total_value = 0  # Total value of all simulations through this node
        self.visits = 0       # Number of visits to this node
        self.move = move      # Move that led to this state
        self.children: List[Node] = []  # Child nodes
        self.parent = parent  # Parent node
    
def rollout(node: Node, current_player: int,player) -> float:
        """
        Simulate a random game from the current node until completion.
        Returns the reward value of the simulation.
        """
        # Check if game is already finished
        winner = node.state.check_winner()
        if winner == player:
            return 1.0
        elif winner == -player:
            return -1.0
        elif winner == 0:  # Draw
            return 0.0
            
        # Get available moves and simulate random play
        moves = node.state.get_available_moves()
        if not moves:
            return 0.0
            
        # Make a random move and continue simulation
        row, col = random.choice(moves)
        new_state = deepcopy(node.state)
        new_state.board[row, col] = current_player
        next_node = Node(new_state, parent=node, move=(row, col))
        return rollout(next_node, -current_player,player)

def backpropagate(node: Node, value: float) -> None:
        """
        Update node statistics on the path from a leaf node to the root.
        """
        while node is not None:
            node.visits += 1
            node.total_value += value
            node = node.parent

def uct_score(node: Node, parent_visits: int, exploration_constant: float = sqrt(2)) -> float:
        """
        Calculate the Upper Confidence Bound for Trees (UCT) score for a node.
        Balances exploitation (first term) and exploration (second term).
        """
        if node.visits == 0:
            return float('inf')
        
        exploitation = node.total_value / node.visits
        exploration = exploration_constant * sqrt(log(parent_visits) / node.visits)
        return exploitation + exploration


def select_best_child(node: Node) -> Node:
        """Select the child node with the highest UCT score."""
        return max(node.children, key=lambda child: uct_score(child, node.visits))

def monte_carlo_tree_search(state: 'TicTacToe', player: int, num_simulations: int = 100) -> Tuple[int, int]:
    
    root = Node(state)
    
    # Main MCTS loop
    for _ in range(num_simulations):
        # Selection: traverse tree until we reach a node that needs expansion
        current = root
        while current.children and all(child.visits > 0 for child in current.children):
            current = select_best_child(current)
        
        # Expansion: if node has been visited and game isn't over, expand by adding all possible child nodes
        if current.visits > 0:
            available_moves = current.state.get_available_moves()
            if available_moves and not current.children:
                for move in available_moves:
                    new_state = deepcopy(current.state)
                    new_state.board[move[0], move[1]] = player
                    current.children.append(Node(new_state, parent=current, move=move))
                current = random.choice(current.children)
        
        # Simulation: perform rollout from current node
        simulation_result = rollout(current, player,player)
        
        # Backpropagation: update statistics for all nodes in path
        backpropagate(current, simulation_result)
    
    # Return best move based on most visited child
    if not root.children:
        return random.choice(state.get_available_moves())
    
    best_child = max(root.children, key=lambda child: child.visits)
    return 0,best_child.move

In [7]:
# Simulate a match between two methods
def simulate_game(method1, method2):
    times_dict = {1:[],-1:[]}
    game = TicTacToe()
    players = [1, -1]
    methods = {1: method1, -1: method2}
    while game.check_winner() is None:
        current_player = players[0]
        start_time=  time.time()
        utility,move = methods[current_player](game, current_player)
        end_time=  time.time()
        times_dict[current_player].append(end_time-start_time)
        game.make_move(*move, current_player)
        players.reverse()
    # game.display()

    return game.check_winner(),times_dict

In [8]:
def test(alg1,alg2,test_num):
    print(f"test has been started for {alg1.__name__} and {alg2.__name__}")
    alg1_wins = 0
    alg2_wins = 0
    tie = 0
    times_avgs = {1:[],2:[]}
    for i in range(test_num):
        winner,times_dict = simulate_game(alg1,alg2)
        times_avgs[1].append(sum(times_dict[1])/len(times_dict[1]))
        times_avgs[2].append(sum(times_dict[-1])/len(times_dict[-1]))
        if winner == 1:
            alg1_wins+=1
        elif winner == -1:
            alg2_wins+=1
        else:
            tie+=1
        print(f"simulation number {i+1}/{test_num} is finished")
    print("test has been finished")
    print("----------------------------------------------")
    return {f"{alg1.__name__} wins":alg1_wins,f"{alg2.__name__} wins":alg2_wins,'Tie':tie,f'{alg1.__name__}_avg':round(sum(times_avgs[1])/len(times_avgs[1]),3),f'{alg2.__name__}_avg' : round(sum(times_avgs[2])/len(times_avgs[2]),3)}
        

In [9]:
def TestAllMethods(test_cases):
   results_data = {
        'Algorithm1': [],
        'Algorithm2': [],
        'Algorithm1_Wins': [],
        'Algorithm2_Wins': [],
        'Tie': [],
        'Algorithm1_Avg': [],
        'Algorithm2_Avg': []
    }
    
    # Run tests and collect results
   for alg1, alg2, test_count in test_cases:
        result = test(alg1, alg2, test_count)
        
        # Extract data from test results
        results_data['Algorithm1'].append(alg1.__name__)
        results_data['Algorithm2'].append(alg2.__name__)
        results_data['Algorithm1_Wins'].append(result[f"{alg1.__name__} wins"])
        results_data['Algorithm2_Wins'].append(result[f"{alg2.__name__} wins"])
        results_data['Algorithm1_Avg'].append(result[f"{alg1.__name__}_avg"])
        results_data['Tie'].append(result['Tie'])
        results_data['Algorithm2_Avg'].append(result[f"{alg2.__name__}_avg"])
   return results_data

In [10]:
import pandas as pd
from typing import Callable, Dict, List

def create_test_results_excel(results_data,output_file: str = 'algorithm_comparison.xlsx'):
   
    
    # Create DataFrame and export to Excel
    df = pd.DataFrame(results_data)
    

    df.to_excel(output_file, sheet_name='Algorithm Comparison', index=False)
    
    # Get workbook and worksheet objects for formatting
    print(f"Results have been exported to {output_file}")


In [11]:
# Main function for testing
if __name__ == "__main__":
       
       test_cases = [
        (minimax, alpha_beta, 1),
        (evaluation_based, minimax, 1),
        (minimax, monte_carlo_tree_search, 30),
        (alpha_beta, evaluation_based, 1),
        (monte_carlo_tree_search, alpha_beta, 30),
        (evaluation_based, monte_carlo_tree_search, 200)
    ]
       resulted_data = TestAllMethods(test_cases)
    #    print (resulted_data)
       create_test_results_excel(resulted_data,)




    

test has been started for minimax and alpha_beta
simulation number 1/1 is finished
test has been finished
----------------------------------------------
test has been started for evaluation_based and minimax
simulation number 1/1 is finished
test has been finished
----------------------------------------------
test has been started for minimax and monte_carlo_tree_search
simulation number 1/30 is finished
simulation number 2/30 is finished
simulation number 3/30 is finished


KeyboardInterrupt: 